In [1]:
import pickle
import pandas as pd
import numpy as np
import re
from IPython.display import clear_output

In [ ]:
DATA_OUT_PATH = 'data/'
DATA_IN_PATH = '../../3.data/'

# 데이터 불러오기

**1. raw 데이터**

In [3]:
f = open('JobKorea_convert.pickle', 'rb')
raw_df = pickle.load(f)
f.close()
raw_df.head()

,company_name,company_url,apply_info,applicant_info,score,score_desc,paper,url
0,(주)경동나비엔,https://www.jobkorea.co.kr/Recruit/Co_Read/C/k...,2021년 상반기 신입 제품·서비스영업,\r\n\r\n 수도권4년\r\n ...,2,전반적으로 아쉬운 자기소개서입니다. 질문에 대한 명쾌한 답이 없다는 점이 본 자기소...,[[자신의 가장 자랑스러운 성취 경험 또는 자신을 가장 힘들게 했던 실패 경험은 무...,https://www.jobkorea.co.kr/starter/PassAssay/V...
1,한국수력원자력(주),https://www.jobkorea.co.kr/Recruit/Co_Read/C/c...,2021년 상반기 신입 해외영업·무역영업,\r\n\r\n 수도권4년\r\n ...,2,전반적으로 호감을 주지 못한 자기소개서입니다. 본 자기소개서의 가장 큰 문제는 지원...,[[본인이 지원한 직무와 관련한 경험 혹은 경력에 대해 기술해 주시기 바랍니다. 다...,https://www.jobkorea.co.kr/starter/PassAssay/V...
2,(주)포스코인터내셔널,https://www.jobkorea.co.kr/Recruit/Co_Read/C/e...,2021년 상반기 신입 수출입·무역사무,\r\n\r\n 고졸\r\n ...,2,일반적으로 많은 지원자들이 실수하는 방식대로 쓴 자기소개서입니다. 자신에 대해 소개...,"[[성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. , \r\n""끈기...",https://www.jobkorea.co.kr/starter/PassAssay/V...
3,㈜하나은행,https://www.jobkorea.co.kr/Recruit/Co_Read/C/h...,2021년 상반기 신입 금융·보험영업,\r\n\r\n 대학원\r\n ...,2,지원자의 가장 큰 문제는 경력은 많으나 지원 회사 및 직무와 연관이 없는 경험들이 ...,"[[자기소개서, \r\n벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력\r""보...",https://www.jobkorea.co.kr/starter/PassAssay/V...
4,조선비즈,https://www.jobkorea.co.kr/Recruit/Co_Read/C/c...,2021년 상반기 신입 기자,\r\n\r\n 서울4년\r\n ...,1,본 자기소개서는 조선비즈라는 매체에 기자로 지원하며 쓴 글입니다. 하지만 글 내용에...,"[[자유양식, \r\n저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 ...",https://www.jobkorea.co.kr/starter/PassAssay/V...


**2. 전처리 데이터**

In [4]:
f = open("JobKorea_preprocess2.pickle", 'rb')
processed_df = pickle.load(f)
f.close()

processed_df.head()

,company_name,paper,url,preprocess1,preprocess2,preprocess3
0,(주)경동나비엔,[[자신의 가장 자랑스러운 성취 경험 또는 자신을 가장 힘들게 했던 실패 경험은 무...,https://www.jobkorea.co.kr/starter/PassAssay/V...,[@@@@@@@@@@@@@@@새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었...,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...
1,한국수력원자력(주),[[본인이 지원한 직무와 관련한 경험 혹은 경력에 대해 기술해 주시기 바랍니다. 다...,https://www.jobkorea.co.kr/starter/PassAssay/V...,"[@@@@@@@@@@@@@@@1) 언제, 어디서 활동했던 경험인지 기술해 주십시오....","[4번의 졸업작품 참여한 경험이 있습니다. 주전공인 정보통신공학, 복수 전공인 영어...","[4번의 졸업작품 참여한 경험이 있습니다. 주전공인 정보통신공학, 복수 전공인 영어..."
2,(주)포스코인터내셔널,"[[성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. , \r\n""끈기...",https://www.jobkorea.co.kr/starter/PassAssay/V...,"[@@@@@@@@@@@@@@@""끈기를 가지고 노력하면 불가능한 것은 없다""$$$$$...","[""끈기를 가지고 노력하면 불가능한 것은 없다""@@@@@@@@@@@@@@@어떠한 일...","[""끈기를 가지고 노력하면 불가능한 것은 없다""@@@@@@@@@@@@@@@어떠한 일..."
3,㈜하나은행,"[[자기소개서, \r\n벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력\r""보...",https://www.jobkorea.co.kr/starter/PassAssay/V...,"[@@@@@@@@@@@@@@@벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력%...","[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력@@@@@@@@@@@@@@@""...","[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력@@@@@@@@@@@@@@@""..."
4,조선비즈,"[[자유양식, \r\n저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 ...",https://www.jobkorea.co.kr/starter/PassAssay/V...,[@@@@@@@@@@@@@@@저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무...,[저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어낸 경험...,[저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어낸 경험...


In [5]:
def convert(contents):
    temp = []
    for content in contents:
        content = content.replace('$$$$$$$$$$$$$$$$$', '\n')
        content = content.replace('%%%%%%%%%%%%%%%', '\n')
        content = content.replace('@@@@@@@@@@@@@@@', '\n')
        temp.append(content)
        
    return temp

processed_df['convert_preprocess3'] = processed_df.apply(lambda x: convert(x['preprocess3']), axis=1)

**3. 테스트 데이터**

In [ ]:
f = open("jokor_0.pickle", 'rb')
test_dic = pickle.load(f)
f.close()

## 사용할 데이터 선택

In [6]:
data_df = processed_df.copy()

# 전처리

## 문장 처리 (맨 뒤에 글자수~, 아쉬운점)

In [11]:
new_contents = []
for idx in range(len(raw_df)):
    #print(idx)
    temp = []
    for contents in raw_df.iloc[idx].paper:
        question, content, point = contents
        # \r => \n
        content = content.replace("\r", "\n")

        # 글자수 제거
        content = re.sub("글\s?자\s?수[ \d자Byte,byte]+", "", content)
        content = re.sub(' +', ' ', content) # 중복 공백 제거
        
        # (200자) 이런거 제거
        content = re.sub("\(\d+자\)", "", content)
        
        if point:
            # 아쉬운 점 제거
            content = re.sub("아\s?쉬\s?운\s?점\s?[\d ]", " ", content)
            
            # 좋은 점 제거
            content = re.sub("좋\s?은\s?점\s?[\d ]", " ", content)
            
            content = re.sub(' +', ' ', content) # 중복 공백 제거
        temp.append(content)

    new_contents.append(temp)
    
raw_df['remove_noused'] = new_contents
raw_df.head()

,company_name,company_url,apply_info,applicant_info,score,score_desc,paper,url,remove_noused
0,(주)경동나비엔,https://www.jobkorea.co.kr/Recruit/Co_Read/C/k...,2021년 상반기 신입 제품·서비스영업,\r\n\r\n 수도권4년\r\n ...,2,전반적으로 아쉬운 자기소개서입니다. 질문에 대한 명쾌한 답이 없다는 점이 본 자기소...,[[자신의 가장 자랑스러운 성취 경험 또는 자신을 가장 힘들게 했던 실패 경험은 무...,https://www.jobkorea.co.kr/starter/PassAssay/V...,[\n\n새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 ...
1,한국수력원자력(주),https://www.jobkorea.co.kr/Recruit/Co_Read/C/c...,2021년 상반기 신입 해외영업·무역영업,\r\n\r\n 수도권4년\r\n ...,2,전반적으로 호감을 주지 못한 자기소개서입니다. 본 자기소개서의 가장 큰 문제는 지원...,[[본인이 지원한 직무와 관련한 경험 혹은 경력에 대해 기술해 주시기 바랍니다. 다...,https://www.jobkorea.co.kr/starter/PassAssay/V...,"[\n\n1) 언제, 어디서 활동했던 경험인지 기술해 주십시오. \n4번의 졸업작품..."
2,(주)포스코인터내셔널,https://www.jobkorea.co.kr/Recruit/Co_Read/C/e...,2021년 상반기 신입 수출입·무역사무,\r\n\r\n 고졸\r\n ...,2,일반적으로 많은 지원자들이 실수하는 방식대로 쓴 자기소개서입니다. 자신에 대해 소개...,"[[성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. , \r\n""끈기...",https://www.jobkorea.co.kr/starter/PassAssay/V...,"[\n\n""끈기를 가지고 노력하면 불가능한 것은 없다""\n\n어떠한 일을 시작하기 ..."
3,㈜하나은행,https://www.jobkorea.co.kr/Recruit/Co_Read/C/h...,2021년 상반기 신입 금융·보험영업,\r\n\r\n 대학원\r\n ...,2,지원자의 가장 큰 문제는 경력은 많으나 지원 회사 및 직무와 연관이 없는 경험들이 ...,"[[자기소개서, \r\n벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력\r""보...",https://www.jobkorea.co.kr/starter/PassAssay/V...,"[\n\n벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력\n""보드전문 개발 벤..."
4,조선비즈,https://www.jobkorea.co.kr/Recruit/Co_Read/C/c...,2021년 상반기 신입 기자,\r\n\r\n 서울4년\r\n ...,1,본 자기소개서는 조선비즈라는 매체에 기자로 지원하며 쓴 글입니다. 하지만 글 내용에...,"[[자유양식, \r\n저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 ...",https://www.jobkorea.co.kr/starter/PassAssay/V...,[\n\n저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어...


## content안에 sub 질문 삭제

In [13]:
# 수작업

## 괄호 처리 (완료된 것 -> preprocessed3 column)

In [238]:
# 사용된 전체 괄호 확인
p = re.compile("\([^\)]*\)")
brackets = []

for cs in new_contents:
    for c in cs:
        brackets.extend(p.findall(c))

# 긴문장 확인하여 자소서 내용이면 내용 놔두고 괄호 삭제
longbrackets = []

for i, bracket in enumerate(brackets):
    if len(bracket) > 100:
        for j, content in enumerate(new_contents):
            for c in content:
                if bracket in c:
                    longbrackets.append(j)
                    break
            else:
                continue
            print(raw_df.iloc[j]['url'])
        print(bracket)
        print()

longbrackets = list(set(longbrackets))
longbrackets.sort()

# 긴문장의 괄호 -> 오타 수작업으로 변경
a1 = ['@@@@@@@@@@@@@@@대학 시절 고등학생의 진로 멘토링을 하며 교육업에 뜻을 품었습니다. 인간의 존재 의미는 일을 통한 사회 공헌에 있다고 생각합니다. 원활한 행정처리로 학생들의 대학생활에 도움을 주고싶습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@가장 일하고 싶은 분야는 국제처입니다. 영어 및 중국어 역량을 바탕으로 교환학생들의 어려움에 귀 기울이고 더 나은 교환학생 생활을 위해 다방면으로 지원방안을 마련하는 도움 되는 직원이 되고 싶습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@상대의 의견을 수용할 수 있는 개방적인 태도입니다. 상대방이 틀린것이 아니라 다른것임을 인지함으로써 상대의 의견을 경청할 수 있고 이를 통해 원활한 커뮤니케이션 및 협업을 이룰 수 있기 때문입니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@아르바이트를 할 당시 오전 출근자로부터 당일 새벽에 연락이 와서 급한 일이 생겼으니 근무시간을 바꿔달라는 요구를 받았습니다. 저는 당시 오전에 건강검진이 예정되어 있었습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@이유도 듣지 못했기에 탐탁지 않았지만 다급해 보이는 목소리에 걱정이 되어 건강검진을 연기하고 수락했습니다. 뒤늦게 들은 이유는 아버지의 임종을 지키기 위해서였습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@배려는 사소하지만 영향은 위대하는 것을 배웠습니다. 그 분은 일에 신경 쓰지 않고 아버지의 마지막 모습을 함께 할 수 있었습니다. 오늘날처럼 밖에서 많은 시간을 보내는 사회인에게는 배려가 더욱 필요합니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@1) 패션 브랜드 입/퇴점 기획%%%%%%%%%%%%%%%직무영역: 기획 %%%%%%%%%%%%%%%역할: 프로젝트 매니저%%%%%%%%%%%%%%%수행내용: 분당, 강남, 송파에 위치한 백화점의 상권 분석 및 고객 라이프스타일 분석을 통해, 각 상권에 최적화 된 패션 브랜드의 입점을 제안 및 상권에 맞지 않는 브랜드의 퇴점 제안 %%%%%%%%%%%%%%%성과: 매출 위주의 입/퇴점 관행을 깬 고객중심적 MD 모델 완성$$$$$$$$$$$$$$$$$2) 영업시스템 설계 %%%%%%%%%%%%%%%직무영역: 기획 %%%%%%%%%%%%%%%역할: 프로젝트 매니저%%%%%%%%%%%%%%%수행내용: 판매사가 상주하지 않는 셀프쇼핑 매장의 매출을 높이기 위한 영업시스템 설계. 고객 구매특성에 따른 상품별 진열방식 변경, 피크타임 분석을 통한 FILL-UP 시스템 도입, 판매사 일일 근무스케줄을 통한 낭비과업 제거%%%%%%%%%%%%%%%성과: 재고일수 50% 감소 및 일매출 30% 성장$$$$$$$$$$$$$$$$$3) 브랜드 관리%%%%%%%%%%%%%%%직무영역: 영업관리 %%%%%%%%%%%%%%%역할: 층 관리자%%%%%%%%%%%%%%%수행내용: 한 아울렛의 아동층 관리자로서 약 30개 브랜드의 매출을 관리, 상권 특성 및 시즌 트렌드를 반영한 프로모션 기획%%%%%%%%%%%%%%%성과: 기획전 매출 전년대비 20% 성장 @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@저는 꾸준히 믿고 있는 신앙 및 현재 다니고 있는 교회는 없지만, 대부분의 종교에 대해 개방적인 태도를 갖고 있습니다. 특히, 제가 인생에서 중요하게 생각하는 가치인 섬김과 봉사는 연세대학교의 기독교 정신과도 일치한다고 생각해, 기독교 문화에 대해서 열린 마음을 가지고 있습니다. 따라서 연세대학교의 교직원으로서 사명감을 가지고 섬김과 봉사의 실천을 위해 노력할 것입니다. @@@@@@@@@@@@@@@ ']
a2 = ['@@@@@@@@@@@@@@@"iCOOP생협그룹이 마주보는 춘추전국시대 뛰어들 각오가 되어있습니다."$$$$$$$$$$$$$$$$$iCOOP생협그룹은 최근 국내산 친환경 농산품 판매촉진을 위한 업무협약 및 홍콩시장에 사업진출에 힘써왔습니다. 이러한 두 측면의 행보는 협동조합업의 미래를 꿰뚫는 핵심 Kewword임을 저 또한 여실히 공감을 했습니다. \'생활\' 속에서 필요한 조합원들의 니즈를 반영하고 회계적 펀터멘털을 탄탄히 하기 위한 방안을 고안해내는 일원이 되고자 지원을 결심하게 되었습니다. $$$$$$$$$$$$$$$$$첫째, 회계자격증들을 취득하며 회계에 대한 기본 개념을 배우면서 업무를 위한 밑거름을 쌓아왔습니다. 또한 재무분석 보고서 및 사업보고서를 살펴보며 직접 수치를 구해보며 실무에 부드럽게 적응할 수 있는 능력을 키웠습니다. $$$$$$$$$$$$$$$$$둘째, 최근 게임회사 인턴과 사무보조아르바이트를 하면서 ms office능력과 사무에티켓을 함양할 수 있었습니다. oooooo에서 근무하며 동향보고서와 테스트케이스 등을 작성하면서 엑셀과 워드를 많이 사용해보았고 사무보조아르바이트는 수출행사 대행사의 실적집계요원과 회계사무소에서 주택담보대출심사 보조를 하였습니다. $$$$$$$$$$$$$$$$$이렇게 쌓은 저의 역량으로 은행/대형마켓/다양한 지역사업 등이 얽혀있는 춘추전국시대에서 "iCOOP 조합원의 요구와 사업을 우선적으로 발 빨리 파악하여, 꼼꼼하고 성실한 회계업무를 처리하기 위해 적극적으로 행동할 것입니다" @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@먼저 손을 내밀줄 아는 사람냄새 나는 人 $$$$$$$$$$$$$$$$$러시아의 표트르 대제는 개혁을 통해 서구문물을 수용했지만 흥선대원군은 쇄국정책으로 일관하였고, 결국 정반대의 결과를 낳고 말았습니다. 자신 그리고 조직의 발전과 화목을 위해선 먼저 한발 다가 설 줄아는 노력이 필수라고 생각합니다. 이는 조합업계에서 요구하는 조합원과의 네트워크구축과 회계리뷰를 하기 위한 커뮤니케이션 필요과정에서 필수적인 역량이라 생각합니다.$$$$$$$$$$$$$$$$$대학교 때 전과를 한 저는 첫 팀플과제가 주어졌는데 교수님께서 하고싶은 사람끼리 조를 짜오라 하셨습니다. 아는 사람이 한명도 없었고 모두가 삼삼오오 팀을 만들고 있었습니다. 아무도 다가오는 이가 없었을 때 문득 먼저 말을 걸고 손을 내밀어보고 싶었습니다. $$$$$$$$$$$$$$$$$인상이 친근해보이는 남학우 부터 시작해서 강의실 곳곳에 혼자 앉아있거나 아직 조를 못짠 것 같은 인원들에게 다가가 같이 해보지 않겠냐고 물었습니다. 그랬더니 모두들 흔쾌히 그러겠다고 답해주었습니다. 이렇게 맺은 4명의 인연들은 졸업때까지 친한사이로 유지할 수 있었습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@"똑같은 유형의 고객문의 해결하기"$$$$$$$$$$$$$$$$$최근oo이라는 게임회사에서 CS 및 어플운영관리 업무를 했었습니다. 그 과정에서 하루에 30~50여건의 문의들어 왔는데 중복되는 유형이 눈에 들어왔습니다. ▶어플과 관련없는 각 게임문의 ▶관련사의 게임문의 ▶이미 날짜가 지난 이벤트 문의 등이 대표적이었습니다. 일일이 동일유형에 답변이 다 나가야 했기 때문에 다른 업무도 있는데 행정력이 낭비되는 것 같았습니다. $$$$$$$$$$$$$$$$$그래서 최근 한달 간의 문의들을 접수된 백오피스에서 유형건으로 엑셀정리를 한 후 , 이러한 문의들에 대해서는 강제적으로 경고창이 뜨거나 해당문구를 박아놓자고 의견을 구해보았습니다. 팀업 무회의시간에 팀장님과 직속 사수는 긍정적으로 검토해보겠다고 하였고 사업팀에게 다음 패치 때 적용하겠다고 피드백이 돌아왔었습니다. $$$$$$$$$$$$$$$$$이렇게 행동했던 이유는 우리 부서에는 해결해줄 수 없는 일임에도 계속 이관처리하는 불편함이 있어서였습니다. 그래서 인턴신분에는 이러한 개선점을 발견할만한 시간이 있었기에 최대한 주도적으로 노력해보았던 경험이었습니다. 앞으로 이러한 행동력을 iCOOP생협그룹에서도 꼭 발휘해보고 싶은 마음이 있습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@"정직하게 평가받지 못한 시험점수"$$$$$$$$$$$$$$$$$4학년 2학기 공공정책과 법 기말평가 때 교수님께서 \'조장이 조원들의 참여점수를 메일로 제출해라\'는 공지를 주었습니다. 가장 열심히 한 인원에게는 5점이였고 불성실한 인원에게는 1점이였으며 중복점수는 불가했습니다. 객관적으로 정직하게 점수배분을 하고 싶었지만 결정이 쉽지 않았고 제 나름대로 불만도 있었습니다. $$$$$$$$$$$$$$$$$조원들이 다들 친한 동기와 형들이었고 발표준비를 제가 가장 많이했기 때문입니다. 또한 메일로만 제출하면 아무도 모르기에 나의 점수를 5점으로 주었고 나머지는 저의 주관적인 생각으로 점수를 부여한 후 제출했습니다. 나의 결정으로 타인의 학점이 관련된 중요한 사항임에도 불구하고 구성원들에게 전달이나 공지과정이 없었던 고 그대로 방학을 맞이한게 지금 생각해도 올바르지 못한 실수였다고 생각합니다. $$$$$$$$$$$$$$$$$앞으로는 친한사이의 경우라도, 상황에 맞는 자의적인 판단을 최대한 배제하고 공적인 업무에서는 관련 절차와 객관적인 업무처리를 하기 위해 노력할 것입니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@"팥빙수 한 그릇"$$$$$$$$$$$$$$$$$여름방학 동안 실내포장마차에서 요리보조 및 서빙경험이 있습니다. 여름철을 맞아, 메뉴에 없었던 팥빙수를 손님들에게 공짜로 서비스 해주기로 했습니다. 반응은 좋았고 손님들 또한 꾸준히 이어졌습니다. 바쁜 탓에 쉬는시간이 될 때마다 저와 또래 아르바이트생들은 쉬는공간에서 직접 팥빙수를 만들어먹었습니다. $$$$$$$$$$$$$$$$$그러다가 문득 에어컨도 나오지 않는 주방 뒷공간에서 설거지만 끊임없이 하고 있는 이모님 생각이 났습니다. 어서 팥빙수를 한 그릇 만들어 가져다 드리자 생각보다 굉장히 좋아하시는 모습이 기억에 남습니다. 한 여름 2개월동안, 나이 또래에 맞는 손님들을 챙기기에 급급해 가장 가까운 주변을 돌아보지 못했구나 라고 강한 울림을 느꼈던 경험이었습니다. $$$$$$$$$$$$$$$$$앞으로도 누군가를 처음 만날 때의 설레임을 꾸준히 유지하여 타인에게 강한 믿음을 주고자 합니다. 그런마음가짐이라면 iCOOP생협그룹 조합원 및 사업관련 이해관계자들과 함께 어울려나갈 수 있다고 생각합니다. @@@@@@@@@@@@@@@ ']
a3 = ['@@@@@@@@@@@@@@@[ 장점 : 조직의 마침표를 찍으며 신뢰를 얻었습니다 ] %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%\'10/5/5\' 제가 참여했던 아르바이트/대외활동/봉사활동 숫자입니다. 20개의 활동이 끝나고 마무리 기념사진에는 항상 저의 얼굴이 찍혀있었습니다. 약속한 근무기간 이전에 그만둔적이 한번도 없었고, 팀프로젝트 활동에서도 개인사정을 핑계로 중도하차해 팀원들에게 피해를 끼치게 하는 일은 절대 없었습니다. $$$$$$$$$$$$$$$$$이번 봄에도 계약기간이 끝났지만 전화가 오시는 구두가게 사장님이 있었고, 작년 겨울에는 적극적으로 토론동아리 활동에 참여해준 것에 대한 감사의편지를 보낸 후배가 있었습니다. $$$$$$$$$$$$$$$$$끝까지 맡은 업무를 다하고 남들이 꺼려하는 일을 개의치 않고 묵묵히 했습니다. 그러자 자연스레 주변사람들의 신뢰의 얻었고 이것은 저의 큰 자산으로 남았습니다. "사실 사회人이라면 책임있는 자세로 신뢰를 얻는 것은 당연할 것입니다" 그러나 너무나 당연하기에 다시 한번 되새기고 싶었습니다. $$$$$$$$$$$$$$$$$특히나, 신뢰의 가치는 넥슨네트웍스 게임서비스 직무에서 큰 빛을 발휘할 것이라고 생각합니다. 고객과의 커뮤니케이션이 끊임없이 이루어져야 하는 업무의 특성상 1순위로 갖추어야 할 덕목이기 때문입니다. $$$$$$$$$$$$$$$$$[ 경험 : "관찰력"을 키운 백화점 판매직 ] $$$$$$$$$$$$$$$$$백화점에서 6개월동안 구두 Sales 단기아르바이트를 하였습니다. 게임업과 유통업은 다르지만 그속에는 고객이라는 공통분모가 존재합니다. 단순히 판매하는 것을 넘어서 고객의 동선과 시즌특수에 따른 판매수요를 예측하고 관찰해보았습니다. $$$$$$$$$$$$$$$$$그에 맞추어 제품 DP도 재배치해보며 고객의 편의를 최대한 돕고자 노력했습니다. 이는 고객의 발걸음을 한번이라도 더 붙잡게 만들었다고 자신합니다. 사후적으로 돕는 것이 아니라 실시간으로 세일즈에 최선을 다했던 것처럼 , 넥슨네트웍스에서도 부지런한 관찰人이 되겠습니다. $$$$$$$$$$$$$$$$$관찰력을 통해서 시즌이 바뀌는 것에 대한 수요를 미리 준비했고, 실시간으로도 고객의 동선파악을 통해 매출을 올리고자 노력했습니다. 운영팀에서도 하나의 게임을 맡게 된다면, 이벤트나 새롭게 출시하는 게임의 런칭에 대한 사전준비 부분, 실시간으로 터지는 이슈에 대해 빠른 리포팅을 통해 문제를 최대한 빨리 해결할 수 있다고 생각합니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%[ 특기 : 문제분석 -≫ 대안제시-≫ 환원으로 이어지는 해결적사고와 숫자적 감각 ] %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%행정학 전공이 게임서비스 직무에 어떤 점이 도움이 될까를 고민해보았습니다. 전공특성상 Issue가 되는 사회현상들을 최대한 중립적인 입장에서 해결해야 하는 과제들을 수행했었습니다. 한 가지의 문제에 꼬리에 꼬리가 물리는 이해관계자들을 보면서 사소한 사안이라도 큰 시야로 바라보는 안목을 키웠습니다. $$$$$$$$$$$$$$$$$또한 통계학 강의를 들으면서 직접 필드에 나가 서베이도 해보고 SPSS프로그램을 통한 DATA검증으로 객관적인 수치를 산출해보았습니다. 그리고 기본OA프로그램과 회계관련 자격증들을 취득하여 숫자적 감각도 함양했습니다. 0 @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@[ 마인드 , 소통. 꼼곰함 게임서비스 직무를 위한 3박자를 갖추었습니다 ] $$$$$$$$$$$$$$$$$다수의 아르바이트와 대외활동을 하면서 남녀노소의 사람들을 대하면서 발생하는 "각 계층의 문제들과 요구사항"을 많이 접하였습니다. 그럴때마다 역량내에서 최선을 다해 도우면서 서비스직무의 기본Mind를 키웠습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%또한 블로그를 운영하면서 "oooooo페스티벌 . ooooo시 SNS홍보단"에 참가하였고, 그 기간동안 상대방이 필요한 정보와 문의사항들에 피드백을 하는 소통창구 역할을 했습니다. $$$$$$$$$$$$$$$$$마지막으로는 매사에 꼼꼼하고 신중한 자세입니다. 현재는 회계법인에서 채권실사업무 아르바이트를 하고 있는데 언제나 변수가 생겨 상황마다 달리 접근해야 할때가 있습니다. 그럴때마다 A4용지에 변경사항을 메모해둔 후 집에와서 워드작업으로 기존의 파일을 수정합니다. 다음 업무에는 그것을 보면서 헷갈리지 않고 빨리 숙달하기 위함입니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%이러한 저의 삼박자가 잘 맞물려 넥슨네트웍스의 게임서비스 직무를 해낼 수 있다고 판단했습니다. $$$$$$$$$$$$$$$$$[ 모든 유저들의 마음을 대변하고 싶습니다 ] -1차적 목표 %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%IT기술이 발달하면서 기존의 온라인게임에서 모바일게임으로 트렌드가 옮겨가면서 게임유저들의 `일회성 소비성향이 짙어지고 있습니다. 그만큼 철새들이 많이 늘어나고 침묵하는 다수가 많이 생겨 나는 것 같습니다. 이메일과 커뮤니티게시판을 통해 불만사항을 요구하는 고객은 해결을 해주면 됩니다. $$$$$$$$$$$$$$$$$하지만 말도 없이 게임을 등져버리는 빙산의 일각 아래에 있는 대다수의 유저들이 더 무섭습니다. 잠수이별로 헤어진 커플은 서로 이유도 모르기 때문에 무엇을 고쳐야 하는지도 모르기 때문입니다. 앞으로 입사하게 된다면 비단, 수면 위로 떠오른 문제와 함께 앞으로 예상되는 위협요소도 관리하는 다재다능한 인재가 되겠습니다. $$$$$$$$$$$$$$$$$[ 더 나은 CRM과 게임시스템 개선을 위해서 ] -2차적 목표%%%%%%%%%%%%%%% %%%%%%%%%%%%%%%업무에 어느정도 적응 한후에 전문가로 성장하기 위한 추가적으로 자기계발을 꾸준히 하고 싶습니다. 페이스북에 보면 LG마케팅팀이나 부산경찰서의 홍보활동 들이 FUN한 요소들이 섞여 재생산되고 퍼트려지면서 소비자들에게 친근하게 다가가고 있습니다. $$$$$$$$$$$$$$$$$이는 홍보효과도 높이고 브랜드 긍정적인 이미지도 함께 가져옵니다. 게임산업 또한 주요Target이 10~30대의 젊은층이 많은만큼 IT공간에서 이루어지는 활동들이 많습니다. 앞으로 점점 더 "즉흥적"이고 "스웨그스러워질" 그들의 입맞에 맞는 고객관리도 연구해 나가고 싶습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%[ K - GAME 의 서비스 전문가 ] - 3차적목표 $$$$$$$$$$$$$$$$$마지막은 K-GAME 의 서비스 전문가가 되는 것이 최종목표입니다. 철 없던 고등학교 시절 , 카드라이더에 빠져 독서실 비용으로 PC방에가 대회준비를 했던 기억이 있습니다. 그때는 단순히 즐거움만을 쫓았다면 지금은 게임산업이 가지는 고부가가치를 아는 청년으로 성장했습니다. 0$$$$$$$$$$$$$$$$$IT강국에 걸맞게 다른 그 어떤 분야보다 게임은 미래를 주도할 유력산업이며 그 속에 넥슨의 게임들은 언제나 리더임을 확신합니다. 20년 뒤 세계를 주도할 K-GAME의 서비스 부문을 종합적으로 컨트롤할 수 있는 스페셜리스트로 성장해 자아실현을 이룰 것입니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@[ 행동으로 보여주고(선조치) -≫ 상대설득하기(후보고) ] $$$$$$$$$$$$$$$$$조선education 입시설명회 보조무를 맡은 적이 있습니다. 입시열기가 뜨거운 만큼 학부모들의 열의 또한 못지 않았습니다. 어느 날 , 한 어머니께서 갑자기 화를 내시면서 자신이 필기했던 노트와 필기구들이 강당의 접이식의자 사이에 빠져 꺼내질 못하니 당장 꺼내달라고 하셨습니다. 왜 갑자기 화를내셨나 살펴보니 여자직원 한명이 그것은 고객의 실수라서 어쩔수없다고만 대응했던것 이였습니다. $$$$$$$$$$$$$$$$$화가 난 고객께서 갑자기 저에게 오시더니 당신이 책임자냐면서 몰아붙이셨습니다. 단순 서브의 신분이었지만 학부모님의 마음을 풀어드리기 위해 행동을 취해야 할 것 같았습니다. 직접 의자 틈에 손을 넣어도 보고 엎드려 꺼내보려고 했지만 여의치 않았습니다. $$$$$$$$$$$$$$$$$그래서 강당 홀의 시설관리팀 전화번호를 알아내 연락을 취해보겠다고 하였고, 1시간 정도 후에 시설관리자가 온다고 전해드렸습니다. 한 시간 후 시설담당자가 도착했고 뒷문개방을 통해 무사히 용품들을 돌려드릴 수 있었습니다. $$$$$$$$$$$$$$$$$그 후에 "시설관리는 우리 소관이 아니라 부득이하게 불편을 끼쳐드려서 죄송하다고" 하며 설득했습니다. 말로만 응대하는 것이 아니라 직접 행동하려는 노력을 보여준 것이 학부모님의 마음을 풀어주는 데 성공할 수 있었던 요인이었습니다. $$$$$$$$$$$$$$$$$[ 2PM 친구와 밥먹기? ] $$$$$$$$$$$$$$$$$상대방의 마음을 헤아려 필요한 도움이 무엇일까? 먼저 한발 다가가 보는 좋은 경험을 한 적이 있습니다. 지난 겨울 , 도서관에서 우연히 중학교 동창을 만났습니다. 이런저런 이야기를 나누다가 경찰공무원을 준비하는 것을 알았고 3번의 낙방을 해서 많이 힘들어했습니다. $$$$$$$$$$$$$$$$$그 친구는 1년6개월동안 카톡도 전혀 사용하지 않고 밥도 혼자 먹었다고 하길래 내일부터 같이 먹자고 했습니다. 그러자 외롭고 심심했는데 고맙다며 환히 웃었습니다. 그렇게 2개월이라는 겨울방학 동안 , 항상 오후 2시에 점심을 같이 먹으며 서로 의지할 수 있는 좋은 친구가 되었고, 친구는 금년 가을에 4수만에 합격해서 꼭 한턱쏘겠다는 카톡메시지가 왔습니다. $$$$$$$$$$$$$$$$$위의 2가지 경험을 통해서 요구하는 바를 정확히 파악하는 것과 고객을 가족처럼 대하는 따듯한 마음씨를 배웠고 넥슨네트웍스에서 적극 발휘하고자 합니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@[ 친구의 배신감 풀어주기 ] - 대화만이 살길이다 $$$$$$$$$$$$$$$$$중고등학교 때부터 함께한 5명의 막역한 친구들 모임이 있습니다. 20살때부터 술을 먹기 시작했고 A라는 친구는 술을 먹으면 정신을 잘 잃어버렸고 B라는 친구는 장난을 잘쳤습니다. $$$$$$$$$$$$$$$$$보통은 계산을 할 때되면 다같이 N분의1 을 했지만 어느 한 날에 , 술을 많이 먹게되었고 B라는 친구가 A친구의 지갑에서 "안낸적도 있지 않냐며" 만원을 더 뺐습니다. 그 후 며칠이 지났고 카톡방에서 갑자기 A친구가 나가버리고 대답이 없었습니다. $$$$$$$$$$$$$$$$$왜그런가 하니 C라는 친구가 사건의 전말을 말해주었고 그 사실을 본인에게 말해주지 않은것에 A가화가 난 것이었습니다. 저 역시 돈을 빼는 행위를 보면서도 , 그 사실을 바르게 전달하지 않고 으레 잘 넘어간 평소의 경우처럼 별 신경을 쓰지 않았습니다. $$$$$$$$$$$$$$$$$결국 B와 저는 왜 별걸도 아닌 것 가지고 사건을 크게 부풀려서 말하는 C에 불만을 품게되고, 감정이 격해진 A와도 빨리 해결을 하고싶었습니다. 카톡과 전화로는 제대로 된 상황설명과 진심을 전달못했기에 직접 만나고 각자의 이야기들을 얘기했습니다. 워낙 친한사이였기에 오히려 사과하기가 힘들었고 서로의 눈을 마주치는게 어색했지만 꼭 거쳐야 할 과정이라 판단했습니다. 좋은점 1$$$$$$$$$$$$$$$$$이후 1주일 지난 후 다시 모든 인원이 화해를 했고 지금은 그 일 이후로 오히려 각자의 특징을 더 알게되어 갈등이 일어나지 않았습니다. $$$$$$$$$$$$$$$$$극복방법은 1. 부딪히는것 2.대화입니다. 이번다툼도 서로의 마음을 왜 몰라줄까? 알아서 알아주겠지 하고 부딪힘이 없었다면 여름여행을 계획하고 있는 지금의 모임이 없을지도 모릅니다. 학창시절 조별과제를 하면서도 발표는 안하고 자료조사만을 맡으려는 인원때문에 흔히 갈등이 일어나게 되는데 , 이때 팀의 성공을 위해선 불편함을 감수하고서라도 부딪히고 대화하는 것입니다.$$$$$$$$$$$$$$$$$말은 쉽지만 누구나 꺼리고 하기 싫어하기 때문에 그만큼 갈등도 많은 것이라 생각합니다. 앞으로 멀리숨어서 기계적인 업무처리를 하지 않고 무조건 부딪히고 살을 맞댈 줄 아는 인간적인 게임서비스 관리자가 되고싶습니다.좋은점 2 @@@@@@@@@@@@@@@ ']
a4 = ['@@@@@@@@@@@@@@@%%%%%%%%%%%%%%%2학년 때까지는 학점 위주로 공부를 했었습니다. 솔직히 그때까지는 깊이 취업에 대한 생각을 하지 않았던 것 같습니다. 3학년이 되어도 정말 불안하고 아무 스펙도 없는 저 자신이 한심하고 무기력하게 느껴졌었습니다. 그래서 목표를 딱하나 정하는 게 시급하여 여러 기업을 조사한 뒤 입사를 꿈꾸게 됐습니다. 그중에서도 제가 친절함을 느꼈었던 신한은행을 목표로 하였고 어느 정도의 스펙을 맞추려 공부를 하였습니다. $$$$$$$$$$$$$$$$$학과 학생회를 하여 리더쉽과 협동심, 그리고 책임감을 배우려 하였습니다. 또 온라인 서포터즈를 통해 스펙관련 Q&A를 하여 OO대학교 뿐만 아니라 여러 학교의 취업준비생들과의 대화를 하여 취업에 대해 알아가고 있었습니다. $$$$$$$$$$$$$$$$$자격증도 두달여간을 준비하여 펀드투자상담사와 파생상품투자상담사를 합격했습니다. 금융관련 자격증을 공부하다 보니 여러기업에 더욱 관심이 생기고 자신감이 붙었습니다. $$$$$$$$$$$$$$$$$가장 중요하게 생각한 것은 였습니다.$$$$$$$$$$$$$$$$$제가 토익이란 것을 대학에 들어와서 알게 되었습니다. 1학년 때 잠깐 학과 스터디를 통해 1주일에 2번씩 선배들에게 토익을 배웠습니다. $$$$$$$$$$$$$$$$$토익에 흥미를 느끼고 있었고 점수도 바닥에서 조금씩 오르고 열의에 가득 차 있었습니다. 그러다 입대를 하게 되 손을 놓고 2학년 때 다시 시작하려니 막막하였습니다. 그때는 이제 전역도 했고 취업에 대해 조금씩 알게 되었기 때문입니다. $$$$$$$$$$$$$$$$$그래서 동기와 같이 학교 동아리 TIME시사 연구회에 들어가 매일매일 TIME잡지 해석 및 발표도 하여 영어공부를 제대로 시작하리라 다짐했습니다. $$$$$$$$$$$$$$$$$그러나 동아리는 저에게 조금 벅찼고 따라가기 힘들었습니다. 혼자서 잡지 해석을 하기에는 너무 실력이 없었고 부담되고 점수도 큰 변화가 없었습니다. $$$$$$$$$$$$$$$$$그래도 뒤처지지 않기 위해 계속 공부해보려 노력했었습니다. 3학년 때 동기 학우들과 스터디를 통해 단어시험과 Part5.6을 1주일에 3번씩 모여 2시간씩 서로 알고 있는 문법과 노하우를 공유했습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%그때는 공부에 집중도 잘되고 점수도 올릴 수 있다는 확신이 들었습니다. 정기토익 점수가 올랐습니다. 그렇게 높은 점수는 아니지만 한 단계 나아갈 수 있을 것 같았습니다. $$$$$$$$$$$$$$$$$좋은결과를 얻진 못했지만 계속 점수가 오를 수 있다는 자신감을 가질 수 있었습니다. 4학년 때 취업준비에 앞서 취업스터디에 들어갔습니다. $$$$$$$$$$$$$$$$$학교 취업지원과에서 주최하는 것으로 전문가 선생님과 같이 각자 맡은 기업의 인재상 및 재무분석, 사업보고서를 통한 기업의 현실태를 조사했습니다. $$$$$$$$$$$$$$$$$뿐만 아니라 경제기사를 스크랩해서 분석하고 경제용어 공부도 하여 금융지식을 쌓았습니다. $$$$$$$$$$$$$$$$$이렇게 준비한 결과를 보듯이 저는 기업인이 되어 미래를 내다보고 준비하여 그에 맞게 기업을 이끌도록 하겠습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@%%%%%%%%%%%%%%%중학교 시절부터 사회교과목에 관심이 많았습니다. 지금 현재 일어나는 현상들과 미래에 더 발전될 모습들은 생각하니 과거에는 어땠을까? 하는 궁금증과 함께 역사를 알아보고 싶은 마음이 생겨 고등학교에 문과를 지원하여 수많은 사회과목들을 배우다 보니 경제에 특히 몰두하게 되었습니다. $$$$$$$$$$$$$$$$$대학 진학에 있어 경제학과를 택하게 되었고 대학생활을 함에 있어 딱 마음먹은 것이 있었습니다. 흔히들 말하는 “놀 때는 놀고 공부할 때는 딱 공부만 하자” 말을 새기며 1학년 때는 솔직히 많이 놀면서 1년을 보냈습니다. $$$$$$$$$$$$$$$$$군대를 갔다 와서 정신 차리고 공부하려고 마음먹었습니다. 복학을 해서 학점위주로 공부를 열심히 하다보니 성적도 잘 나오고 국가장학금제도가 생겨 장학금을 4학년까지 거의 전액을 받았었습니다. $$$$$$$$$$$$$$$$$그리고 2학년이 되어 넉넉하지 않은 집안 형편을 생각해서 부모님에게 용돈을 받지 않으려고 근로장학생을 매학기 신청하여 거의 2년간제 스스로 돈을 벌어 학교생활을 했었습니다. $$$$$$$$$$$$$$$$$꾸준히 노력하는 으로 저는 2년간 를 하였습니다. 입사 후 저의 자신있는 모습과 부모님에게 효도를 할 수 있는 기회를 갖고 싶습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@%%%%%%%%%%%%%%%모든 일을 되도록 긍정적으로 받아들이고 마음의 여유를 가지고 생활하려 합니다. 하지만 가끔은 마음의 여유를 갖는다는 것이 지나쳐 게으름으로 이어지기도 했습니다. $$$$$$$$$$$$$$$$$그러나 이제 저는 학생이 아닌 사회인의 신분으로 새로운 시작을 위한 출발점에 서 있습니다. 더 이상 게으름을 피우기보다는 마음의 여유를 가지되 몸소 실천하는 사람이 되고자 합니다. $$$$$$$$$$$$$$$$$그리고 저는 사교적이고 원만한 대인관계를 유지하며 모든 사람들에게 잘 대하고 잘 보이려고 하는 경향이 있습니다. 다만 모든 사람에게 쓴 소리를 못하고 화를 내지 못합니다. $$$$$$$$$$$$$$$$$이런 점은 스트레스만 쌓이고 저에게 불리해서 고치려고 하고 있습니다. 대신 이러한 인맥으로 다른 사람들에게도 친절하고 따뜻하게 다가가 좋은 관계를 보이도록 하겠습니다. $$$$$$$$$$$$$$$$$기업의 고객을 대할때는 진심을 담아 성심성의껏 모시도록 하겠습니다.$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%삶에 있어 교만한 것은 자기 자신 안에 갇혀 있어 빛을 보지 못한다고 생각합니다. 겸손한 태도로 한발 한발 나아가면 반드시 스스로에게 득이 생긴다고 믿습니다. $$$$$$$$$$$$$$$$$너무 나서지 않고 또 너무 주눅들지 않고 적당하게, 그리고 고객을 마음을 사로 잡을수 있도록 겸손하고 따뜻한 자세로 고객을 최우선순위로 두고 먼저 생각하도록 하겠습니다. $$$$$$$$$$$$$$$$$이러한 태도로 고객에게 항상 친절하고 따뜻한 모습을 보이도록 하겠습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@%%%%%%%%%%%%%%%작년일이었습니다. 후배의 갑작스러운 연락이 왔는데 할 말이 있다며 저를 부르는 것이었습니다. 그 당시가 시험기간이었는데 시험족보와 또 잘 모르는 게 있다며 가르쳐 달라고 했습니다. $$$$$$$$$$$$$$$$$처음엔 후배여서 챙겨주고 싶은 마음에 제가 아는 대로 다 가르쳐 줬습니다. 그리고 고맙다며 같이 밥도 먹었는데 그 뒤로는 연락이 오질 않았습니다. $$$$$$$$$$$$$$$$$그러려니 하다가 기말고사 때 연락이 오고 가르쳐 주고 나니 다시 연락이 오지 않았습니다. 2학기가 되어 중간고사 때 다시 연락이 와서 참다참다 결국 화를 냈습니다. $$$$$$$$$$$$$$$$$후배가 죄송하다고 그랬지만 변명을 계속하였습니다. 그때는 너무 괘씸하고 이기적이라는 생각이 너무 박혀있어서 얘기조차 하고 싶지 않았습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%그 뒤로 연락이 오질 않다가 후배한테 화를 낸 것이 미안해 먼저 연락해서 대화를 하고 사과를 했습니다. 그 후배는 오히려 자기가 죄송하고 또 고맙다고 해주었습니다. $$$$$$$$$$$$$$$$$그 후에는 친해져서 자주 연락하는 사이가 되고 가끔 술도 마시는 절친한 사이가 되었습니다.$$$$$$$$$$$$$$$$$처음부터 제가 말을 했었더라면 그런 후회도 드는데 갈등은 서로의 불만사항을 이야기하면 풀린다고 생각합니다. 마음에 담아두지 않고 솔직하게 대화하는 것이 중요하다고 생각합니다. $$$$$$$$$$$$$$$$$이런 선후배나 동기간의 갈등이 생길 수 있기에 저는 [고객과의 커뮤니케이션]을 통해 불만사항을 최소화하고 [고객의 말에 진심으로 귀 기울여 신뢰감을 형성]하겠습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@%%%%%%%%%%%%%%%대학생활을 하면서 잘했다고 생각하는 경험이 두가지가 있습니다.$$$$$$$$$$$$$$$$$첫째, 라는 학교 내 동아리에서 영어 및 토익공부를 한 것입니다.$$$$$$$$$$$$$$$$$전역하고 나서 처음으로 영어공부를 시작한 계기가 되었습니다. 이 동아리를 통해 저보다 훨씬 영어에 능통하고 잘하는 사람이 많았습니다. $$$$$$$$$$$$$$$$$1년간 하면서 ‘이 친구들과 계속 하다보면 영어를 잘할 수 있겠지’라는 생각을 하였습니다. 그런데 늘지 않는 실력을 보며 제 자신의 나태함, 게으름을 알게 되었습니다. $$$$$$$$$$$$$$$$$스스로 공부하여 노력하여도 실력이 늘까말까한데 저는 동아리만 믿고 제 스스로 노력을 하지 않았던 것입니다. $$$$$$$$$$$$$$$$$그나마 다행이라 할만한 것은 ‘이대로는 안되겠다’며 스스로를 돌아보게 되고 제 자신이 최선을 다해 좋은 결과를 얻도록 하겠습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%둘째, 인터넷 카페 활동을 통해 취업정보를 공유하고 서로의 고민거리를 상담해주는 서포터즈를 하였습니다. $$$$$$$$$$$$$$$$$이 서포터즈는 2달여간 활동을 하는데 활동내역은 게시글과 댓글을 남겨 취준생들을 도와주는 것입니다.이 활동을 통해 저도 정보를 얻고 제가 아는 정보를 상대방에게도 최대한 상세히 알려주는 것입니다. $$$$$$$$$$$$$$$$$제가 직접 기업사이트에 들어가서 자격증여부, 스펙, 기업정보를 알아보는 등 적극적인 활동을 하였습니다. %%%%%%%%%%%%%%% %%%%%%%%%%%%%%%마침 서포터즈활동 2달 후 수료증을 받은 이후에 활동이 뜸해졌습니다. 그러다 채용공고가 뜨고 다들 바쁘게 준비하는 가운데서 저는 뒤늦게 알아낸 경우가 많았습니다. $$$$$$$$$$$$$$$$$그때마다 후회하고 스스로 안타까웠습니다. 더 적극적으로 알아보고 미리 준비하는 자세를 길러 후회하지 않도록 대비할 것입니다. @@@@@@@@@@@@@@@ ']
a5 = ['@@@@@@@@@@@@@@@저는 해외 각지를 돌아다니며 일을 할 수 있는 건설사 취업을 목표로 하고 있습니다. 그 중에서도 현대건설은 택한 이유는 끝없는 도전정신과 성과에 대해서 공정한 보상을 한다는 것이 매력적이기 때문입니다. 입사한다면 직접 해외를 돌아다니며 현장의 경험을 쌓고 싶습니다. $$$$$$$$$$$$$$$$$그리고 도면과 현장여건을 같이 체크하며 공부를 하고 싶습니다. 그리고 보다 큰 그림을 그리며 설계를 하고 싶습니다. 직접 현장경험이 있다면 설계업무를 맡을 때 이득이 있을 것이라 생각합니다.$$$$$$$$$$$$$$$$$다른 부서도 마찬가지겠지만 이런 현장업무는 협동심이 없으면 안 된다고 생각합니다. 따라서 자기주장보다는 타인의 의견을 들을 줄 아는 능력이 더 필요하다고 생각합니다. 이런 능력은 실제로 다른 사람들과 같이 작업을 하면서 기를 수 있다고 생각하며 밑에서 서술할 공모전 경험을 통하여 이런 소양을 쌓을 수 있었습니다. 이 공모전을 통해 협동심과 열정적이고 자기주도적인 자세를 키웠습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@환경이나 생활이 힘들다면 이를 극복하기 위해선 그 안에서 즐거움을 찾아야 한다고 생각합니다. 전 예전 비행기 수화물을 내리는 곳에서 아르바이트를 한 적이 있습니다. 비행기가 내리면 그 안의 손님들의 짐과 각종 쓰레기 음식물들을 옮기는 일이었습니다. 육체적으로 굉장히 힘든 일이라 원래 근무하던 직원들도 3일 근무 1일 휴식이 기본이었습니다. $$$$$$$$$$$$$$$$$그 곳에서 계속 일만하다가는 제풀에 지쳐 일을 그만둘 것 같아서 다른 방안을 찾아보고자 하였습니다. 따라서 소장님께 건의하여 근무시간에 음악을 들을 수 있게 하고 잠깐씩 쉬는 시간을 지정하여 흡연을 할 수 있게 하였습니다. $$$$$$$$$$$$$$$$$그 결과 전 개강 전까지 아르바이트를 꾸준히 할 수 있었고 등록금에도 적지 않은 보탬이 되었습니다. 해외에 나가서도 예상치 못한 어려움이 있을 수 있고 또 힘들어서 주저앉고 싶어질 수도 있을 것입니다. 하지만 그런 상황에서도 즐거움을 찾고 없으면 만들어서 일을 해나가겠습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@기계공학 부분에서 중요한 것은 모델의 구조적 해석이라고 생각합니다. 이를 위하여 기구학이나 시스템동역학, 계측공학 과목을 통해 본드그래프와 매틀랩의 기본적인 사용법을 배웠습니다. 그리고 전산제도 과목을 통하여 간단한 모델링 방법과 CAD의 기초적인 사용방법등을 익혔습니다. 물론 실제로 회사에서 사용하는 프로그램과는 다르겠지만 기본적인 역량을 갖추고 최대한 빨리 프로그램을 배우는데 도움이 될 것이라 생각합니다. $$$$$$$$$$$$$$$$$그리고 전공에서 배운 실험실습을 통하여 설계에서만 끝나는 것이 아니라 실제로 레고를 조립하거나 모델을 분석을 해보는 일을 하였습니다. 회사라는 곳은 결국 조직이라고 생각합니다. 조직에서 중요한 것은 구성원과 얼마나 함께 잘 일 할 수 있는지와 조직을 사랑하는 마음이라고 생각합니다. 회사가 아무리 힘들고 어렵더라도 그 길을 함께 걷고 싶습니다. @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@현대건설은 설계부분을 현대엔지니어링으로 맡기는 것으로 알고 있습니다. 하지만 직접 현장에서 다양한 경험을 하고 보다 큰 그림을 그릴 줄 알며 설치 여건 등을 고려할 능력이 되는 사람이 설계를 맡는 다면 더 효율적으로 일을 할 수 있을 것 이라 생각합니다. $$$$$$$$$$$$$$$$$오로지 설계만 하던 사람들이 실제 현장의 설치 여건을 고려하지 못하고 설계를 한다면 업무 효율이 좋지 않을 것이라 생각합니다. 따라서 시공에서 일을 하다가 설계를 맡는 것도 나쁘지 않으며 이런 부분을 현대건설내부에서 해결할 수 있다면 더 나아질 수 있을 것이라 생각합니다. 굳이 현대엔지니어링에 다 의존하는 것보다 내부적으로도 일을 처리할 수 있는 부서가 있다면 업무효율을 더 개선할 수 있지 않을까 생각합니다.$$$$$$$$$$$$$$$$$하지만 자신이 노력하면 현장은 도면 및 현장여건을 크로스 체킹하면서 공부도 가능하고 현장은 설계업무 보다 중요한 다양한 경험을 접할 수 있기 때문에 나중에 큰 그림을 그리고 싶으시다면 시공에 몸을 담구는 것도 나쁘지는 않습니다 @@@@@@@@@@@@@@@ ', '@@@@@@@@@@@@@@@2011년에 전 도전정신을 키울 수 있는 경험을 하였습니다. 추억을 쌓을 수 있는 일이 없을까 고민하던 저희의 눈에 들어온 것은 간단한 공모전 이었습니다. 이 공모전에서 작게나마 가작을 받고나자 되든 안 되든 일단 열정을 가지고 무엇인가를 해봐야한다는 사실을 배웠습니다. 이 경험은 다른 공모전을 준비하고자 할 때 도움이 되어 2014년에 타 회사에서 진행한 자동차 에너지 효율 개선 공모전에 나갈 수 있는 발판이 되었습니다. 이번엔 제가 직접 공모전이 없나 나서서 찾아보고 교수님께 여쭤서 공모전을 찾아 준비할 수 있었습니다.$$$$$$$$$$$$$$$$$이 공모전에서 한 가지 주제만을 가지고도 여러 가지의 해결방법이 있다는 사실을 보면서 문제 해결을 위해선 한 분야만이 아니라 여러 분야에서 힘을 모아야 한다는 사실을 깨달았습니다. 그리고 실제로 제품을 만드는 것은 단지 프로그램으로 모델링하는 것과는 다르다는 것을 실감했습니다. 그래도 저희는 서로의 의견을 들으며 협업하여 저번보다 좋은 결과인 우수상을 얻었습니다. @@@@@@@@@@@@@@@ ']

change = [a1, a2, a3, a4, a5]

# 수작업한 내용으로 바꾸기
for i, c in zip(longbrackets, change):
    new_contents[i] = c
    
# 모든 괄호 제거
p = re.compile("\([^\)]*\)")
brackets = []

for i, contents in enumerate(new_contents):
    for j, content in enumerate(contents):
        new_contents[i][j] = p.sub('', content)
        
for cs in new_contents:
    for c in cs:
        brackets.extend(p.findall(c))
        
print(len(brackets))
brackets = list(set(brackets))
print(len(brackets))
brackets # 빈 리스트가 결과로 o

https://www.jobkorea.co.kr/starter/PassAssay/View/193572?Page=120&OrderBy=0&FavorCo_Stat=0&Pass_An_Stat=0
(고객 구매특성에 따른 상품별 진열방식 변경, 피크타임 분석을 통한 FILL-UP 시스템 도입, 판매사 일일 근무스케줄을 통한 낭비과업 제거%%%%%%%%%%%%%%%성과: 재고일수 50% 감소 및 일매출 30% 성장$$$$$$$$$$$$$$$$$3)

https://www.jobkorea.co.kr/starter/PassAssay/View/167581?Page=170&OrderBy=0&FavorCo_Stat=0&Pass_An_Stat=0
(oooooo에서 근무하며 동향보고서와 테스트케이스 등을 작성하면서 엑셀과 워드를 많이 사용해보았고 사무보조아르바이트는 수출행사 대행사의 실적집계요원과 회계사무소에서 주택담보대출심사 보조를 함 )

https://www.jobkorea.co.kr/starter/PassAssay/View/166673?Page=194&OrderBy=0&FavorCo_Stat=0&Pass_An_Stat=0
((=≫관찰력을 통해서 시즌이 바뀌는 것에 대한 수요를 미리 준비했고, 실시간으로도 고객의 동선파악을 통해 매출을 올리고자 노력했습니다. 운영팀에서도 하나의 게임을 맡게 된다면, 이벤트나 새롭게 출시하는 게임의 런칭에 대한 사전준비 부분, 실시간으로 터지는 이슈에 대해 빠른 리포팅을 통해 문제를 최대한 빨리 해결할 수 있다고 생각합니다.)

https://www.jobkorea.co.kr/starter/PassAssay/View/146665?Page=238&OrderBy=0&FavorCo_Stat=0&Pass_An_Stat=0
(2학년 때까지는 학점 위주로 공부를 했었습니다. 솔직히 그때까지는 깊이 취업에 대한 생각을 하지 않았던 것 같습니다. 3학년이 되어도 정말 불안하고 아무 스펙도 없는 저 자신이 한심하고 무기력하게 느껴졌었습니다. 그래서 목표를 딱하나

[]

## 사용되는 특수문자 종류

In [12]:
# 종류 확인
p = re.compile("[^\d\sa-zA-Zㄱ-힣]+")
noword = []

for contents in data_df.convert_preprocess3:
    for content in contents:
        noword.extend(p.findall(content))
        
noword = list(set(noword))
len(noword)

498

### 어떻게 쓰이는지 확인

In [10]:
for token in noword[1:]:
    print(token)
    for i, contents in enumerate(data_df.convert_preprocess3):
        for content in contents:
            if token in content:
                print(i)
                print(content)
                print()
                print("*"*60)
    input()
    clear_output(wait=False)

’’
1283
"인턴이지만 하나의 브랜드를 담당하다"
컨설팅 회사에서 고객사인 OOOOOO의 20억 글로벌 고객 대상으로 통계 분석과 고객 프로세스 개선 제안 업무를 했으며 빠른 적응력과 강한 책임감으로 인턴이지만 OOOOO라는 하나의 브랜드를 담당하는 성과를 이루었습니다. 입사하기 전 데이터를 추출할 때 네티자라는 프로그램에서 SQL 언어를 사용한다는 말을 듣고 학부 시절 배웠던 MYSQL 공부와 성능 효율이 높은 쿼리가 무엇인지에 대한 공부에 몰두하였습니다. 결과적으로 업무에 빠르게 적응해 예정에 없었던 고객 분석 업무를 맡으며 고객사에 직접적인 분석 서비스를 제공 할 수 있었습니다. 인턴 기간 동안 가장 열정적으로 수행한 업무는 구매 데이터를 분석한 ‘Indepth Report’를 작성하는 일이었습니다. 프로모션이나 멤버십과 관련된 다양한 레포트를 작성하며 아쉬웠던 점은 회사에 통계 프로그래밍을 활용한 분석 기법이 도입되지 않았다는 점이었습니다. ‘강한상품 경험률’’을 주제로 데이터를 분석할 당시 R을 도입해 연관성 분석을 하였고 향상도와 신뢰도가 높은 규칙을 적용해 CROSS-SELLING을 유도하는 전략을 제안했습니다. 새로운 통계 프로그래밍을 사용해 고객사에게 깊이 있는 레포트를 제공할 수 있었고 긍정적인 피드백을 받을 수 있었습니다. 
이러한 고객의 구매 데이터로 인사이트를 찾아내 더 나은 전략을 수립하는 직무에 적성과 흥미를 느꼈고, 저의 다양한 통계 분석 능력을 활용해 GS SHOP 고객에게 세분화된 추천 상품과 전략을 제시하고 싶습니다. 

************************************************************
3828
[글로벌 살림꾼]
제가 생각한 유통업체의 기본은 ‘고객이 가장 필요한 것을 언제, 어디서나, 편하게 살 수 있어야 된다.’’입니다. 하지만 이 기본에 ‘저렴한 가격이 합해진다면 얼마나 경쟁력이 생겨날까?’라는 생각이 문득 들었습니다. 이미 국내에서 소비자들이 가장 만족하고 있는 브랜드인 G

KeyboardInterrupt: Interrupted by user

### 특수문자 처리

#### 버트 vocab에 없는 문자 처리

In [14]:
# 자소서에서 사용되는 특수문자 확인
p = re.compile("[^\d\sa-zA-Zㄱ-힣]")
noword_t = []

for contents in data_df.convert_preprocess3:
    for content in contents:
        noword_t.extend(p.findall(content))
        
noword_t = list(set(noword_t))

# 버트 vocab 불러오기
f=open("vocab.txt", encoding='utf-8-sig')
vocab = f.read()
f.close()
vocab_li = vocab.split('\n')

# 버트 vocab에 없는 특수문자 추출
noBertTokens = []
for n in noword_t:
    if n not in vocab_li:
        noBertTokens.append(n)
print(noBertTokens)

['¸', '\U000f0852', 'Å', '＇', '➂', 'ᄁ', '⓷', '＃', '\U000f0853', '➀', '˙', '♪', '➁', '\uf09e', '∞', '⓶', 'Ω', '⓸', '⓵', '₃', '\u200b', '♬']


In [15]:
# 안쓰이는 특수문자 처리
noBertToken_dic = {}
noBertToken_dic['⓵'] = '1'
noBertToken_dic['♬'] = ''# 제거
noBertToken_dic['➂'] = '3'
noBertToken_dic['\U000f0853'] = ''# 제거
noBertToken_dic['⓷'] = '3'
noBertToken_dic['₃'] = '3'
noBertToken_dic['¸'] = ''
noBertToken_dic['Å'] = 'A'
noBertToken_dic['♪'] = ''
noBertToken_dic['\u200b'] = ''
noBertToken_dic['＃'] = '#'
noBertToken_dic['➀'] = '1'
noBertToken_dic['➁'] = '2'
noBertToken_dic['∞'] = '무한'
noBertToken_dic['⓶'] = '2'
noBertToken_dic['＇'] = ''
noBertToken_dic['Ω'] = 'o'
noBertToken_dic['⓸'] = '4'
noBertToken_dic['\uf09e'] = ''
noBertToken_dic['˙'] = '‧'
noBertToken_dic['\U000f0852'] =''
noBertToken_dic['ᄁ'] = '까'

In [16]:
def remove_tokens(contents):
    temp = []
    for content in contents:
        for token in noBertToken_dic.keys():
            content = content.replace(token, noBertToken_dic[token])
        temp.append(content)
        
    return temp

data_df['remove_token'] = data_df.apply(lambda x: remove_tokens(x['convert_preprocess3']), axis=1)

In [17]:
# 확인해보기
for t in noBertTokens:
    print("사용되는 특수문자:  ", t)
    for z, contents in enumerate(data_df.remove_token):
        for content in contents:
                if t in content:
                    print(z)
                    print(content)
    input()
    clear_output(wait=False)
            

## 짧은 자소서 제거 (150글자 이하)

In [19]:
for i, contents in enumerate(data_df.remove_token):
    temp = []
    
    for j, content in enumerate(contents):
        if len(content) < 149:
            print(i, content)
            print()
            data_df.loc[i, 'remove_token'].pop(j)

## 영문 자소서 제거

In [23]:
# 한글, 알파벳 비율로 알파벳 비율이 절반 이상 많을 경우 해당 CONTENT 제거
eng = re.compile('[a-zA-Z]')
kor = re.compile('[ㄱ-힣]')
ratio = 0.6
deleted_idx_eng = []

for i, contents in enumerate(data_df.remove_token):
    temp = []
    
    for j, content in enumerate(contents):
        engWordList = eng.findall(content)
        korWordList = kor.findall(content)
        
        if len(korWordList) == 0:
            print(content)
            print()
            data_df.loc[i, 'remove_token'].pop(j)
            
        elif len(engWordList) == 0:
            continue
        else:
            if  len(engWordList) / (len(engWordList) + len(korWordList)) > ratio:
                print(content)
                print()
                data_df.loc[i, 'remove_token'].pop(j

## 글이아닌 자소서 제거 - 문장 분류 후 진행

In [49]:
# content에서 마지막 4글자 중에 '다' or '요'의 비율이 적은것 삭제
# 띄어쓰기가 이상한 것도 있음
# 아쉬운점 좋은점 안없어진 것도 있음

cut_sentence = []
delet_content_nosentence = []
recheck = []

count_ = 0
ratio = 0.3

for i, contents in enumerate(data_df.remove_token):
    temp = []
    
    for j, content in enumerate(contents):
        sentences = content.split('\n')
        sentences = [sentence.strip() for sentence in sentences if sentence != '']
        
        total = len(sentences)
        count = 0
        for sentence in sentences:
            if ('다' in sentence[-4:]) or ('요' in sentence[-4:]):
                count += 1
        
        if (count / total) < ratio:
            count_ += 1
            print(content)
            print()
            data_df.loc[i, 'remove_token'].pop(j)

#             # 수작업 진행시 코드
#             # 1: 데이터 이상으로 문장 중간이 짤린 데이터
#             # 2: 글로 구성되지 않은 content
#             # any: 정상 or 다시 확인할 것
#             x = input()
#             if x == '1':
#                 cut_sentence.append((i, j))
#             elif x == '2':
#                 delet_content_nosentence.append((i,j))
#             else:
#                 recheck.append((i,j))
#             clear_output(wait=False)

print(count_)

0


## 소제목 분류

**소제목에 사용될 수 있는것**\
" "\
{ }\
[ ]\
< >\
▶▶\
◆ ◆\
“ ”\
‘ ’\
확인해 볼 특수 문자: ＇ ＂  ` ˙  ´ ′ 

In [106]:
# p = re.compile('(^\"[^\n\"]+\"[^\n]*)')
# p2 = re.compile('(^\[[^\n\]]+\][^\n]*)') # 다시 살펴보기
# p3 = re.compile('(^▶▶[^\n(◀◀)]+◀◀[^\n]*)')
# p4 = re.compile('(^◆[^\n(◆)]+◆[^\n]*)')
# p5 = re.compile('(^<<[^\n(>>)]+>>[^\n]*)') # 없음
# p6 = re.compile('(^“[^\n(”)]+”[^\n]*)') # 다시보기
# p7 = re.compile('(^<[^\n(>)]+>[^\n]*)') # 다시보기
# p8 = re.compile('(^\'[^\n\']+\'[^\n]*)') # 아주 가끔 있는데 생각해보기
# p9 = re.compile('(^‘[^\n’]+’[^\n]*)')
# p10 = re.compile('(^`[^\n`]+`[^\n]*)') # 1089처럼 엔터처리가 이상한것도 있다.

# 완벽한것
p1 = re.compile('(^▶▶[^\n(◀◀)]+◀◀$)')
p2 = re.compile('(^◆[^\n(◆)]+◆$)')
p3 = re.compile('(^<[^\n(>)]+>$)') # 다시보기
p4 = re.compile('(^\'[^\n\']+\'$)') # 아주 가끔 있는데 생각해보기

# 긴문장 확인해볼 것
p5 = re.compile('(^\"[^\n\"]+\"$)')
p6 = re.compile('(^\[[^\n\]]+\]$)') # 다시 살펴보기
p7 = re.compile('(^“[^\n(”)]+”$)') # 다시보기
p8 = re.compile('(^‘[^\n’]+’$)')
p9 = re.compile('(^`[^\n`]+`$)') # 1089처럼 엔터처리가 이상한것도 있다.

reCompilers = [p1, p2, p3, p4, p5, p6, p7, p8, p9]
check = []

max_length = 80 # 최대 문자열 길이 (해당 길이 이상은 자소서의 글의 일부)

for contents in data_df.remove_token:
    temp = []
    
    for content in contents:
        sub_temp = []
        sentences = content.split('\n')
        
        for sentence in sentences:
            smallSubjects = []
            for reCompiler in reCompilers:
                smallSubjects.extend(reCompiler.findall(sentence.strip()))

            # 문장 길이 설정 일정값 이상 긴문장은 거르는 거르기
            smallSubjects = [smallSubject for smallSubject in smallSubjects if len(smallSubject) < max_length]
            
            sub_temp.extend(smallSubjects)
        if len(sub_temp) != 0:
            print(sub_temp)
        temp.append(sub_temp)
        
    check.append(temp)
print(len(check))

['"끈기를 가지고 노력하면 불가능한 것은 없다"']
['"보드전문 개발 벤처기업부터 제조 공장 자동화, 금융개발까지"']
['"불확실성의 ‘위기’ 속에서도 상승 ‘분위기’로"']
['"기업의 사회적 책임: 긴즈버그를 통해서"']
['"음지의 양지화, 그 뒤에는 SNS의 다크웹화"', '"AI를 활용한 해결전망"']
['“크기는 작지만 큰 역할”', '“맡은 일에 대한 책임감과 열정”']
['“소통을 통한 목표 달성능력”', '“타인의 입장을 고려한 제안”']
['“신차개발 품질은 초기에 확보해야 한다.”']
['"뭐든지 최고가 목표"']
['"여러 제약들을 발판삼아 일어나자"']
['"발로 뛰며 트렌드를 만들어내는 Specialist는 바로 나!"']
['"소비자만족 1위 통합형 멤버십"', '"통합되지 않은 APP"']
['"성공적인 세미나 준비하기"']
['"창의력 캠프 최우수상 달성"']
['"근본적인 해결책 제시"']
['"상호 존중의 중요성"']
['"다방면으로 사고하는 힘"']
['"내 이웃이란 마음으로 가스 안전을 알리겠습니다"']
['"필요하지 않은 과정은 없다."']
['"공동 목표를 위해 먼저 배려하는 마음"']
['"끈기를 가지고 안전한 가스 사용에 도움이 되겠습니다"']
['"따듯한 공감의 나눔"']
['"솔선수범은 주변에도 전해진다"']
['"분석과 해결방안"']
['"화학, 에너지 그 이상의 가치"']
['"공정개선경험과 전공지식"']
['"공정데이터 분석으로 생산량 20%향상"']
['"보이지 않는 곳에서 최선을 다하는 인재"']
['"친환경 발전기술을 세계 시장에 전파"']
['"동기를 부여하는 소통의 자세"']
['"사소한 행동이 불러온 긍정적 효과"']
['"정직함과 성실함"']
['"눈높이를 맞춘 면담"']
['"고객만족을 위한 세심한 배려"']
['"사소한 행동이 불러온 긍정적 효과"']
['"동기를 부여하는 소통의 자세"']
['"엉킨 실타래는 생각보다 단순하게 풀린다."']
['"신뢰성"']
['"내실을 굳게 다진 소재

In [103]:
# for i, contents in enumerate(data_df.remove_token):
def remove_smallSubject(i, contents):
    temp = []
    for j, content in enumerate(contents):
        checkers = check[i][j]
        for checker in checkers:
            content = content.replace(checker, "")
        temp.append(content)
            
    return temp
            
data_df['remove_smallSubject'] = data_df.apply(lambda x: remove_smallSubject(x.name, x.remove_token), axis=1)

""



## 엔터로 되어있는 부분 공백으로 분리

In [112]:
splitedContents_by_enter = []

for contents in data_df.remove_smallSubject:
    temp = []
    for content in contents:
        splitedContent = content.split('\n')
        splitedContent = [sentence.strip() for sentence in splitedContent if len(sentence) != 0]
        temp.append(splitedContent)
        
    splitedContents_by_enter.append(temp)
    

In [ ]:
# 개수 확인
c = 0
for i in splitedContents_by_enter:
    for j in i:
        c += 1
        
print(c)

In [117]:
f = open("preprocessed_data.pickle", "wb")
pickle.dump(splitedContents_by_enter, f)
f.close()

# 기타

## 특수문자 통일 - 진행중 멈춤

In [6]:
# 길이가 1인 문자열 확인
for token in noword:
    if len(token) == 1:
        print(token)
        input()
        clear_output(wait=False)

In [7]:
#따옴표 -> " " 통일,  ' 이거는 : 문자'문자 아닐때만 바꾸는 걸로
type1 = "‘ ＇ ＂ ’ ` ˙ ” ´ “ ' \" ′"
type1 = type1.split()

#숫자
type2 = "➀ ² ➁ ₂ ⓶ ① ② ⑤ ⑥ ® ③ ⓵ ④ < ⓸ ₃ ⓷ ➂ Ⅱ Ⅳ"
type2 = type2.split()

#불렛
type3 = "● • ○"
type3 = type3.split()

#괄호
type4 = "《 } 「 ） > ＞ [ 』 『 》 ) ≫ （ 」 ＜ ［ ( {"
type4 = type4.split()

#화살표
type5 = "⇒ →"
type5 = type5.split()

#모르겠음 
type6 = "@ Å \ ＃ - □ ； … $ 󰡓 ! / 金 樂 ▷ ․ é ~ ◈ ▶ ° # : – 。 ． ? ｍ & 󰡒 ＆ ◆ | % ℃ ‧ ： ∼ +  ^ ▣ ± · ★ μ ; _ × Ⅰ * ∙ = ∞ ■ ， ¸"
type6 = type6.split()

In [8]:
# type별로 처리방법 정리
for t in type6:
    print("사용되는 특수문자:  ", t)
    for contents in data_df.convert_preprocess3:
        for content in contents:
                if t in content:
                    print(content)
    input()
    clear_output(wait=False)
            

사용되는 특수문자:   
[책임감으로 꼼꼼하게 업무를 수행하겠습니다]
저는 인턴십을 통해 팀에서 각자의 역할이 있다는 것을 인지하고, 사소한 일이라도 자신에게 주어진 업무라면 책임감을 가지고 완벽하게 수행하여야 팀 전체의 관점에서 더 향상된 결과를 얻을 수 있다는 것을 배웠습니다. 
작년 여름 미국 소재 재간접헤지펀드운용사인 ****에서 인턴십을 하며 업무를 수행했던 경험은 제게 특별한 기억으로 남아있습니다. 그 중에서도 펀드 설립을 위해 20개가 넘는 하위펀드의 상품 발행 모집서를 읽고 최소 투자금청약환매사항 등의 계약조건을 정리하는 일은 제게 도전적인 일로 다가왔습니다. 
전체 펀드 설립 프로세스를 볼 때 제가 맡은 업무는 아주 일부에 불과하지만, ‘내가 맡은 일이 펀드 설립에 있어 꼭 필요한 업무이다’ 라고 생각하며 제일 복잡한 펀드의 발행 모집서를 3일 간 꼬박 손으로 정리해가며 정독했습니다. 
이를 통해 저는 그 펀드의 구조를 파악하게 되었고, 이러한 구조적 사고는 다른 펀드를 이해하는 데에도 많은 도움이 되어 업무를 효율적으로 수행할 수 있었으며 펀드 설립도 성공적으로 진행되었습니다. 
단순히 ‘관리자’라는 생각을 벗어나 저 또한 지점에 소속된 일원이라는 생각으로 업무에 임하겠습니다. 이러한 저의 책임감으로 제가 맡은 지점의 예산과 지표를 꼼꼼하게 관리하여 실적 목표치를 달성하겠습니다. 


KeyboardInterrupt: Interrupted by user

In [ ]:
# type6 처리
'@' # 유지
'Å' A
'\\' # 유지 or ,
'＃' # 유지
'-' # 화살표 불렛, 이름의 중간 등 많이 사용 유지가 좋을 듯
'；' ;
'…' # 삭제해도 좋을 듯
'$' # 유지하는게 좋을 듯
'\U000f0853' # 제거
'!' # 유지
'/' # 유지 
'金' # 한자 어떻게할까??
'樂' # 한자 어떻게할까??
'․' # 가운데 점으로 많이사용 선.후배 (기본 .과 다름)
'é' e
'~' # 유지
'°' # 온도에서 쓰임
'#' # 유지
':' # 유지
'–' # 처리가 필요할듯
'。' .
'．' .
'?' # 유지
'ｍ' m
'&' # 유지
'\U000f0852' # 제거
'＆' &
'|' # 제거
'%' # 유지
'℃' # 어떻게 할까 위에 도라 같이 처리하면 좋을 듯
'‧' # 가운데 점으로 사용 이게 좋을 듯?
'：' :
'∼' ~
'+' +
'\uf09e' # 제거
'^' ^
'±'
'·'
'★'
'μ'
';'
'_'
'×'
'Ⅰ'
'*'
'∙'
'='
'∞'
'■'
'，'
'¸'

In [ ]:
# type5 처리
type4_dic = {}

type4_dic['⇒'] =  '->'
type4_dic['→'] =  '->'

In [13]:
# type4 처리
type4_dic = {}
type4_dic['《'] = '<'
type4_dic['》'] = '>'

type4_dic['「'] = '<'
type4_dic['」'] = '>'

type4_dic['＞'] = '>'
type4_dic['＜'] = '<'

type4_dic['『'] = '<'
type4_dic['』'] = '>'

type4_dic['（'] = '('
type4_dic['）'] = ')'

type4_dic['≫'] = '>' # 화살표로도 쓰임 (-≫)
type4_dic['≪'] = '<'
    
type4_dic['［'] = '['
type4_dic['］'] = ']'


# # 안해도 될것
# ')' )
# '(' ( # 이거쓰이는거 지워도 될듯?
# '}' }
# '{' {
# '>' >
# '[' [

In [ ]:
# type3 처리
'●'  # 불렛 포인트로 사용됨
'•'  # 선.후배 이럴때 사용됨
'○'  # 이름 가릴때 사용 (가끔 01, 02, 03, 숫자에쓰임 -> 이거만 지워주면 가능)
'▲'  # 불렛 포인트로 사용
'□'  # 불렛으로 사용
'▷' # 불렛으로 사용
'◈' # bullet 사용
'▶' # 불렛 사용, 소제목에도 2개로된것 사용되므로 소제목 먼저 처리하는 것이 좋을 듯
'◆' # 불렛으로 사용 소제목으로도 사용되므로 소제목 먼저 처리하는 것이 좋을 듯
'▣' # 

In [ ]:
# type2 처리
type2_dic = {}
type2_dic['➀'] = 1
type2_dic['➁'] = 2
type2_dic['➂'] = 3
type2_dic['⓵'] = 1
type2_dic['⓶'] = 2
type2_dic['⓷'] = 3
type2_dic['⓸'] = 4
type2_dic['①'] = 1
type2_dic['②'] = 2
type2_dic['③'] = 3
type2_dic['④'] = 4
type2_dic['⑤'] = 5
# 라틴 숫자
['Ⅱ'] = 2
['Ⅳ'] = 4

## 어떻게 처리할까
# # 첨자
# ['²'] = 윗첨자(제곱을 나타내는 것) -> 어떻게 처리할까
# ['₂'] = 아래첨자 2 ->어떻게
# ['₃'] = 아래첨자 2 ->어떻게
# # 제거
# ['®'] = 동그라미안에 R? 회사이름 나타네는거같은데 삭제 가능할 듯 1번쓰임
# ['<'] = 이거는 사용하는 특수문자
